### Flash PL bitstream onto FPGA

In [ ]:
from typing import Final;
from pynq import allocate, Overlay;
from vpu_driver import VPUAccelDriverIP;
from vpu_overlay import VPUAccelOverlay;
from PIL import Image as pil;
import numpy as np;
import matplotlib.pyplot as plot;
import cv2;

# Constants regarding the programmable logic properties

HW_BITSTREAM_FILE : Final = "hw_overlay.bit";
HW_DMA_IP_NAME : Final = "axi_dma_0";
HW_VPU_IP_NAME : Final = "vpu_accel_0";

# Laad de bitstream en systeemlayout

system = VPUAccelOverlay(HW_BITSTREAM_FILE);
system.check_version();

# Laad systeemcomponenten in PL

dma = getattr(system, HW_DMA_IP_NAME);
vpu = getattr(system, HW_VPU_IP_NAME);
vpu.check_integrity();

### Load input image

In [ ]:
# Laad input image

image = cv2.imread("0a0a0b1a-7c39d841.png");

plot.imshow(image[:,:,::-1]); # bgr read

### Allocate SDRAM space & copy image to there

In [ ]:
# Wijs SDRAM toe waar wij de afbeelding willen plaatsen

height, width, channels = image.shape

in_buffer = allocate(shape=(height, width, 3), dtype=np.uint8, cacheable=1)
out_buffer = allocate(shape=(height, width), dtype=np.uint8, cacheable=1)

# Kopieer de afbeelding naar deze ruimte

in_buffer[:] = image

### Activate the HLS Image Processing IP Core

In [ ]:
# Geef een commando aan de digitale logica via de AXI-Lite poort

vpu.register_map.in_width = width
vpu.register_map.in_height = height

dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

In [ ]:
%%time
vpu.write(0x00, 0x81) # write AP_START and AP_AUTOSTART bits

dma.sendchannel.wait()
dma.recvchannel.wait()

In [ ]:
cv2.imwrite("0a0a0b1a-7c39d841.out.png", out_buffer)
plot.imshow(out_buffer, cmap="gray");

In [ ]:
del in_buffer
del out_buffer